In [6]:
import subprocess
import math
from pydub import AudioSegment


def extract_audio_from_video(video_path, audio_path):
    command = ["ffmpeg", "-i", video_path, "-vn", audio_path,]
    subprocess.run(command)
    
def cut_audio_in_chunks(audio_path, chunk_minutes, chunk_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_seconds = chunk_minutes * 60 * 1000
    chunks = math.ceil(len(track) / chunk_seconds)
    for i in range(chunks):
        start_time = i * chunk_seconds  
        end_time = (i+1) * chunk_seconds
        chunk = track[start_time:end_time]
        chunk.export(f"{chunk_folder}/chunk_{i}.mp3", format="mp3")

In [ ]:
# extract_audio_from_video(
#     "./files/wilcox.mp4",
#     ".files/wilcox.mp3"
# )

In [7]:
import openai
import glob
        
def transcribe_chunks(chunk_folder, destination_file):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    for file in files:
        with open(file, "rb") as audio_file, open(destination_file, "a") as text_file:
            transcript = openai.audio.transcriptions.create(
                model="whisper-1", 
                file=audio_file,
            )
            text_file.write(transcript.text) 
            
transcribe_chunks("./files/chunks", "./files/transcript.txt",)